In [ ]:
import os
import sys
import gc
import random
from tqdm import tqdm
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as la

sys.path.insert(0, '../../../methods')
from l1_svd import *

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
rmse_list1 = []
rmse_list2 = []

reg_snr = {-20:1.3063, -15:1.1387, -10:1.28e-4, -5:4.13e-6, 0:1.45e-5, 5:1.94e-5, 10:3.29e-5, 15:1.94e-5,20:7.46e-6, 25:7.25e-6, 30:1.63e-5}

for snr in snr_list:
    # load data
    data = np.load(f'../../../data/experiment_2/scenario_1/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    signals = [s['signals'] for s in data]
    sensor_positions = [s['sensor_pos'] for s in data]

    del data
    gc.collect()

    truths = np.stack(truths).squeeze(2)

    # apply l1-svd algorithm
    preds = np.zeros((len(signals),4))
    reg = reg_snr[snr]

    for i in tqdm(range(len(signals)), leave=False):
        y = signals[i]
        M = y.shape[0]
        N = 4
        res = 1
        sensor_pos = sensor_positions[i]

        phi = np.arange(30, 150+res, res)
        theta = 90
        A = np.zeros((M,phi.shape[0]),dtype=complex)
        for k in range(phi.shape[0]):
            A[:,k] = np.exp(-2j*np.pi*(sensor_pos[:,0] * np.sin(np.deg2rad(theta)) * np.cos(np.deg2rad(phi[k])) +
                                  sensor_pos[:,1] * np.sin(np.deg2rad(theta)) * np.sin(np.deg2rad(phi[k])) +
                                  sensor_pos[:,2] * np.cos(np.deg2rad(theta)))).reshape(-1,)

        preds[i,:], spectrum = l1_svd(y,N,A,phi,res,reg)
        preds[i] = np.sort(preds[i])
        truths[i] = np.sort(truths[i])

        if snr==10 and i==0:
             np.save(f'../../../results/experiment_2/scenario_1/l1-svd_spectrum_{truths[0]}deg.npy', spectrum)

    rmse = np.sqrt(np.mean((truths-preds)**2))
    rmse_list1.append(rmse)

    print(f"snr {snr}dB, T {1000}, test-rmse {rmse:.4f}")

    np.save(f'../../../results/experiment_2/scenario_1/l1-svd_preds_snr{snr}_t1000.npy', preds)
    np.save(f'../../../results/experiment_2/scenario_1/l1-svd_truths_snr{snr}_t1000.npy', truths)

for T in T_list:
    # load data
    data = np.load(f'../../../data/experiment_2/scenario_1/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    signals = [s['signals'] for s in data]
    sensor_positions = [s['sensor_pos'] for s in data]

    del data
    gc.collect()

    truths = np.stack(truths).squeeze(2)

    # apply l1-svd algorithm
    preds = np.zeros((len(signals),4))
    reg = reg_snr[-10]

    for i in tqdm(range(len(signals)), leave=False):
        y = signals[i]
        M = y.shape[0]
        N = 4
        res = 1
        sensor_pos = sensor_positions[i]

        phi = np.arange(30, 150+res, res)
        theta = 90
        A = np.zeros((M,phi.shape[0]),dtype=complex)
        for k in range(phi.shape[0]):
            A[:,k] = np.exp(-2j*np.pi*(sensor_pos[:,0] * np.sin(np.deg2rad(theta)) * np.cos(np.deg2rad(phi[k])) +
                                  sensor_pos[:,1] * np.sin(np.deg2rad(theta)) * np.sin(np.deg2rad(phi[k])) +
                                  sensor_pos[:,2] * np.cos(np.deg2rad(theta)))).reshape(-1,)

        preds[i,:], spectrum = l1_svd(y,N,A,phi,res,reg)
        preds[i] = np.sort(preds[i])
        truths[i] = np.sort(truths[i])

    rmse = np.sqrt(np.mean((truths-preds)**2))
    rmse_list2.append(rmse)

    print(f"snr -10dB, T {T}, test-rmse {rmse:.4f}")

    np.save(f'../../../results/experiment_2/scenario_1/l1-svd_preds_snr-10_t{T}.npy', preds)
    np.save(f'../../../results/experiment_2/scenario_1/l1-svd_truths_snr-10_t{T}.npy', truths)

np.save('../../../results/experiment_2/scenario_1/l1-svd_rmse1.npy', rmse_list1)
np.save('../../../results/experiment_2/scenario_1/l1-svd_rmse2.npy', rmse_list2)

In [ ]:
# plot rmse values
plt.figure()
plt.plot(snr_list, rmse_list1, '-o')
plt.title("rmse values for different snr levels")
plt.xlabel("snr (dB)")
plt.ylabel("rmse (deg)")
plt.legend(['l1-svd'])
plt.yscale("log")
plt.grid()

plt.figure()
plt.plot(T_list, rmse_list2, '-o')
plt.title("rmse values for different snapshot numbers")
plt.xlabel("T")
plt.ylabel("rmse (deg)")
plt.legend(['l1-svd'])
plt.yscale("log")
plt.grid()